In [1]:
%load_ext autoreload
%autoreload 2

import numpy

from robotools import liquidhandling
from robotools import evotools

`robotools` can automatically track the composition of liquids. This is useful if you're preparing a dilution series, or a complex mixture of different components.
With the `Labware.composition` property, you can inspect the relative composition of your liquids.

To use this feature, make sure that you define the correct `initial_volumes` on all of your labwares:

In [2]:
# Troughs
water = liquidhandling.Labware('water', 1, 1,
                               min_volume=1000, max_volume=25000,
                               initial_volumes=25000, virtual_rows=4)
cgxii = liquidhandling.Labware('cgxii', 1, 1,
                               min_volume=1000, max_volume=25000,
                               initial_volumes=25000, virtual_rows=4)
iptg = liquidhandling.Labware('IPTG', 1, 1,
                               min_volume=1000, max_volume=25000,
                               initial_volumes=25000, virtual_rows=4)
# Eppis with inoculum
inoculum = liquidhandling.Labware('inoculum', 4, 1, min_volume=20, max_volume=1500, initial_volumes=100)
# FlowerPlate (only the first 3 columns for simplicity)
cultures = liquidhandling.Labware('cultures', 4, 3, min_volume=0, max_volume=1000)

# Then proceed to do regular liquid handling steps
with evotools.Worklist() as wl:
    # add media to all wells
    wl.distribute(
        cgxii, 0,
        cultures, cultures.wells,
        volume=760
    )
    # add different amounts of IPTG
    wl.transfer(
        iptg, iptg.wells[:,[0]*3],
        cultures, cultures.wells,
        volumes=numpy.array([
            [4]*3,
            [8]*3,
            [12]*3,
            [16]*3,
        ])
    )
    # inoculate
    wl.transfer(
        inoculum, inoculum.wells[:,[0]*3],
        cultures, cultures.wells,
        volumes=20
    )
    # fill up to 800 with water
    wl.transfer(
        water, water.wells[:,[0]*3],
        cultures, cultures.wells,
        volumes=800-cultures.volumes
    )

The `Labware.composition` dictionary now describes the relative amounts of each component in the liquid. By multiplying it with the current volumes, we get back the µl-fractions:

In [3]:
cultures.composition['inoculum'] * cultures.volumes

array([[20., 20., 20.],
       [20., 20., 20.],
       [20., 20., 20.],
       [20., 20., 20.]])

To compute the final concentration of a component, just multiply its relative amount with the original concentration:

In [4]:
cultures.composition['IPTG'] * 100 # mM

array([[0.5, 0.5, 0.5],
       [1. , 1. , 1. ],
       [1.5, 1.5, 1.5],
       [2. , 2. , 2. ]])

One can also inspect the relative composition of an individual well:

In [5]:
cultures.get_well_composition('C02')

{'cgxii': 0.95, 'IPTG': 0.015, 'inoculum': 0.025, 'water': 0.01}

As expected, they sum up to 1.0:

In [6]:
sum(cultures.get_well_composition('C02').values())

1.0

Still reading? Go and try it yourself! Maybe with a dilution series?